# Indentifying the subject of comments

## EDA

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import zipfile
# # unzip image file
# local_zip = '/content/drive/MyDrive/Intent_analysis.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('/content/drive/MyDrive/Intent_analysis')
# zip_ref.close()

In [ ]:
# see what folders are in path
!ls "/content/drive/MyDrive/Intent_analysis"

test.csv  train.csv


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Intent_analysis/train.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Intent_analysis/test.csv')

In [ ]:
train.head()

,id,comment,product_id,product_title_fa,category_id,category_title_fa,is_buyer,price_value,fake_originality,warranty,size,discrepancy,flavor_odor,expiration_date
0,1215666.0,ارزششو داشت,2802959,جوراب مردانه فیرو پلاس مدل FP200 مجموعه 4 عددی,9456,جوراب و ساق مردانه,True,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1183149.0,یه دستگاه کاربردی فقط ای کاش طرز استفادشو میزد...,1894596,دستگاه سفید کننده دندان دنتال وایت مدل DT-1244,5745,مسواک برقی,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1011730.0,خیلی خوب و ارزان,323729,کتاب بهترین قصه ملانصرالدین کلک مرغابی اثر مهر...,5918,کتاب چاپی,True,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1093325.0,اندازش خیلیییییی بزرگ بود,1431242,نوار بهداشتی تافته مدل Thin بسته 7 عددی,6528,نوار بهداشتی,True,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1111983.0,خیلی بد بود و میچسبید به دست,1902220,ژل بازی کد shb57,6083,ابزار شوخی,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test.head()

,id,comment,product_id,product_title_fa,category_id,category_title_fa,is_buyer
0,1130083,پیشنهاد میکنم,723251,پایه نگهدارنده شارژر موبایل مدل Stop charge,81.0,پایه نگهدارنده گوشی و تبلت,True
1,1051036,اول یکی خریدم دیدم خیلی خوبه\r\nخداروشکر نظری ...,623643,کابل تبدیل USB به microUSB تسکو مدل TC 50 طول ...,80.0,کابل و مبدل,True
2,1240179,نسبت به قیمت بصرفه هست,1552547,دستبند مدل S261-A,9541.0,دستبند زنانه و مردانه,True
3,1197397,داخل توضیحات نوشته درب عقب باز میشه ولی اینطور...,1545300,ماشین بازی طرح وانت نیسان مدل جونیور,6055.0,ماشین,True
4,1212378,خیلی زود ازبین میره روی پوت نمی مونه اصلا از خ...,81003,کرم پودر تیوبی مای شماره 202,6462.0,کرم پودر,True


In [ ]:
train.shape

(224130, 14)

In [ ]:
test.shape

(56033, 7)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224130 entries, 0 to 224129
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 224129 non-null  float64
 1   comment            224127 non-null  object 
 2   product_id         224129 non-null  object 
 3   product_title_fa   224129 non-null  object 
 4   category_id        224129 non-null  object 
 5   category_title_fa  224129 non-null  object 
 6   is_buyer           224129 non-null  object 
 7   price_value        224129 non-null  float64
 8   fake_originality   224129 non-null  float64
 9   warranty           224129 non-null  float64
 10  size               224129 non-null  float64
 11  discrepancy        224129 non-null  float64
 12  flavor_odor        224129 non-null  float64
 13  expiration_date    224128 non-null  float64
dtypes: float64(8), object(6)
memory usage: 23.9+ MB


In [ ]:
train.isnull().sum()

id                   1
comment              3
product_id           1
product_title_fa     1
category_id          1
category_title_fa    1
is_buyer             1
price_value          1
fake_originality     1
warranty             1
size                 1
discrepancy          1
flavor_odor          1
expiration_date      2
dtype: int64

In [ ]:
# drop all rows with any NaN and NaT values
train = train.dropna()

In [ ]:
train.isnull().sum()

id                   0
comment              0
product_id           0
product_title_fa     0
category_id          0
category_title_fa    0
is_buyer             0
price_value          0
fake_originality     0
warranty             0
size                 0
discrepancy          0
flavor_odor          0
expiration_date      0
dtype: int64

In [ ]:
train.shape

(224125, 14)

In [ ]:
test.isnull().sum()

id                   0
comment              3
product_id           0
product_title_fa     0
category_id          0
category_title_fa    0
is_buyer             0
dtype: int64

## Text preprocessing

Let's define a function to read file that contains stopwords.

In [ ]:
# define a function to read file
def readFile(filename):
  fileObj = open(filename, 'r') # open the file in read mode
  words = fileObj.read().splitlines() # puts the file into an array
  fileObj.close()
  return words

In [ ]:
stopwords = readFile('/content/drive/MyDrive/AI-Internship/persian_stopwords_kharazi.txt')
print(f'length of stop words : {len(stopwords)}')

length of stop words : 1370


Let's install and import libraries.

In [ ]:
!pip install parsivar

     |████████████████████████████████| 36.2 MB 62 kB/s 
     |████████████████████████████████| 1.5 MB 50.7 MB/s 
  Created wheel for parsivar: filename=parsivar-0.2.3-py3-none-any.whl size=36492971 sha256=2a82661b442230839d0633d8142710e75fbdd49a513fcb8f195fce0dd02ebccf
  Stored in directory: /root/.cache/pip/wheels/ae/67/7a/49cbf08f64d3f76a26eceaf0e481a40e233f05d4356875cbed
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449922 sha256=6227603f533ae37302427f38a10853a34cff2bcc328e69ace743940fe219b4a1
  Stored in directory: /root/.cache/pip/wheels/48/8b/7f/473521e0c731c6566d631b281f323842bbda9bd819eb9a3ead
Successfully built parsivar nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
# Loading NLTk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# NLP libraries
import parsivar
from parsivar import Normalizer, Tokenizer, FindStems, POSTagger, FindChunks

# plot
import matplotlib.pyplot as plt

import pandas as pd
import string
import re
import numpy as np
np.random.seed(0)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# define a function to delete '&' in the middle of verb stemming
def delete_and(word):
  idx = word.find("&")
  if idx!=-1:
    word = word[:idx]
  return word

Let's define a function for text preprocessing.

In [ ]:
def data_preprocessing(review, stopwords):
  try:

    if type(review) == str:
      # replace half-space with ' '
      review =  re.sub('\u200c', ' ',review)

      # Normalizing the text
      # First we should normalize text in order to convert persian numbers into english numbers then
      # with following function (filter) delete them
      # Because, some comments are pinglish, we should set pinglish_conversion_needed = True
      normalizer = Normalizer(pinglish_conversion_needed=True) 
      review = normalizer.normalize(review)

      # because after normalization appear some '/u200c', we should replace them with space
      review =  re.sub('\u200c', ' ',review)

      # delete english characters and numbers from sentences
      review = filter(lambda x: x in string.whitespace or x not in string.printable, review)
      review = ''.join(ch for ch in list(review))
    
      if review != ' ':
        # word tokenization
        tokenizer = Tokenizer()
        words = tokenizer.tokenize_words(review)

        # stemming 
        stemmer = FindStems()
        review = [stemmer.convert_to_stem(word) for word in words]

        # we should delete '&', because after stemming we have '&' in between of our verbs
        review = [delete_and(word) for word in review]

        # remove stop words
        words_without_stopword = filter(lambda x: x not in stopwords, review)
        words_without_stopwords = list(words_without_stopword)

        # join words in preprocessed review
        review = ' '.join(words_without_stopwords)
    
      return review

  except TypeError:
    print(review)
    raise

In [ ]:
! pip install pandarallel

  Created wheel for pandarallel: filename=pandarallel-1.5.2-py3-none-any.whl size=18398 sha256=c989faa0593a5e59c171c246053d0764fc7931d13f44eb2b596d5edc75fb0446
  Stored in directory: /root/.cache/pip/wheels/b5/6d/51/9ece2eaf007ea3f7fb0ce053c5773b2eb0d308887da3af12c1
Successfully built pandarallel


In [ ]:
from pandarallel import pandarallel

In [ ]:
# initialize pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


This cell takes about 3 hours.

In [ ]:
train['pre'] = train['comment'].parallel_apply(lambda text: data_preprocessing(text, stopwords))

test['pre'] = test['comment'].parallel_apply(lambda text: data_preprocessing(text, stopwords))

In [ ]:
#Save dataframe
# !mkdir -p /content/drive/MyDrive/Digi_preprocessed
# train.to_pickle('/content/drive/MyDrive/Digi_preprocessed/train_data')
# test.to_pickle('/content/drive/MyDrive/Digi_preprocessed/test_data')

In [ ]:
#load the preprocessing dataframe
train = pd.read_pickle(r'/content/drive/MyDrive/Digi_preprocessed/train_data')
test = pd.read_pickle(r'/content/drive/MyDrive/Digi_preprocessed/test_data')

In [ ]:
test.head()

,id,comment,product_id,product_title_fa,category_id,category_title_fa,is_buyer,pre
0,1130083,پیشنهاد میکنم,723251,پایه نگهدارنده شارژر موبایل مدل Stop charge,81.0,پایه نگهدارنده گوشی و تبلت,True,پیشنهاد
1,1051036,اول یکی خریدم دیدم خیلی خوبه\r\nخداروشکر نظری ...,623643,کابل تبدیل USB به microUSB تسکو مدل TC 50 طول ...,80.0,کابل و مبدل,True,خرید دید خوبه نظری دراز بسنجمش ماه دید عالی دو...
2,1240179,نسبت به قیمت بصرفه هست,1552547,دستبند مدل S261-A,9541.0,دستبند زنانه و مردانه,True,نسبت قیمت بصرفه
3,1197397,داخل توضیحات نوشته درب عقب باز میشه ولی اینطور...,1545300,ماشین بازی طرح وانت نیسان مدل جونیور,6055.0,ماشین,True,توضیحات نوشته درب میشه اینطورنبود وجنسشم لاکه ...
4,1212378,خیلی زود ازبین میره روی پوت نمی مونه اصلا از خ...,81003,کرم پودر تیوبی مای شماره 202,6462.0,کرم پودر,True,ازبین میره پوت نم مونه خرید راضی


In [ ]:
test.isnull().sum()

id                   0
comment              3
product_id           0
product_title_fa     0
category_id          0
category_title_fa    0
is_buyer             0
pre                  3
dtype: int64

We should remove undesired columns. We need just comment column as a feature.

In [ ]:
train.drop(labels=['id', 'comment', 'product_id', 'product_title_fa', 'category_id' , 'category_title_fa', 'is_buyer'], axis=1, inplace=True) # remove comment column

In [ ]:
test.drop(labels=['id', 'comment', 'product_id', 'product_title_fa', 'category_id' , 'category_title_fa', 'is_buyer'], axis=1, inplace=True)

In [ ]:
train.to_csv('/content/drive/MyDrive/Digi_preprocessed/train.csv', index=False)

test.to_csv('/content/drive/MyDrive/Digi_preprocessed/test.csv', index=False)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Digi_preprocessed/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Digi_preprocessed/test.csv')

In [ ]:
train.head(20)

,price_value,fake_originality,warranty,size,discrepancy,flavor_odor,expiration_date,pre
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ارزششو
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,دستگاه کاربردی طرز استفادشو زد اینقد دنبال کار
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ارزان
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,اندازش خیلیییییی
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,چسبید دست
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,عالی قیمت مناسب فروش خرید
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,مصطفوی عالیه
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,عرض پرده توضیحات زده متر دوتا پرده عرض متر عرض...
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,بهداشت ماسک خوبه


As we can see, we have some rows with 'NaN' values, so we should replace them with space in train and test dataset.

In [ ]:
for i in range(test.shape[0]):
  if type(test.pre[i]) != str:
    test.pre[i] = " "

In [ ]:
for i in range(train.shape[0]):
  if type(train.pre[i]) != str:
    train.pre[i] = " "

In [ ]:
test.isnull().sum()

pre    0
dtype: int64

In [ ]:
train.head(20)

,price_value,fake_originality,warranty,size,discrepancy,flavor_odor,expiration_date,pre
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ارزششو
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,دستگاه کاربردی طرز استفادشو زد اینقد دنبال کار
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ارزان
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,اندازش خیلیییییی
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,چسبید دست
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,عالی قیمت مناسب فروش خرید
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,مصطفوی عالیه
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,عرض پرده توضیحات زده متر دوتا پرده عرض متر عرض...
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,بهداشت ماسک خوبه


In [ ]:
test.head()

,pre
0,پیشنهاد
1,خرید دید خوبه نظری دراز بسنجمش ماه دید عالی دو...
2,نسبت قیمت بصرفه
3,توضیحات نوشته درب میشه اینطورنبود وجنسشم لاکه ...
4,ازبین میره پوت نم مونه خرید راضی


## Feature extractio with TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

tf = TfidfVectorizer(min_df=5)
X_train_tfidf = tf.fit_transform(train['pre'])
X_test_tfidf = tf.transform(test['pre'])

print(f'X_train_tfidf shape : {X_train_tfidf.shape}')
print(f'X_test_tfidf shape : {X_test_tfidf.shape}')

X_train_tfidf shape : (224125, 9936)
X_test_tfidf shape : (56033, 9936)


In [ ]:
train1 = train.copy()
y_train = train1.drop(labels=['pre'], axis=1) # remove useless columns

In [ ]:
y_train

,price_value,fake_originality,warranty,size,discrepancy,flavor_odor,expiration_date
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
224120,0.0,0.0,0.0,0.0,0.0,0.0,0.0
224121,1.0,0.0,0.0,0.0,0.0,0.0,1.0
224122,0.0,0.0,0.0,0.0,0.0,0.0,0.0
224123,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Train the Random Forest classifier model

In this problem we have multi label classification, so we should use MultiOutputClassifier on the random forest model.

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

# xgb = xgb.XGBClassifier(bjective='binary:logistic')
rf = RandomForestClassifier()

multilabel_model = MultiOutputClassifier(rf)


In [ ]:
# fit the model
multilabel_model.fit(X_train_tfidf, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                  

In [ ]:
pred = multilabel_model.predict(X_test_tfidf)

In [ ]:
pred

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
pred.shape

(56033, 7)

In this problem, we have 7 labels in output, so we should define a list for each class and append the value of that label to its list.

In [ ]:
col0 = []
col1 = []
col2 = []
col3 = []
col4 = []
col5 = []
col6 = []

for i in range(pred.shape[0]):
  col0.append(int(pred[i][0]))
  col1.append(int(pred[i][1]))
  col2.append(int(pred[i][2]))
  col3.append(int(pred[i][3]))
  col4.append(int(pred[i][4]))
  col5.append(int(pred[i][5]))
  col6.append(int(pred[i][6]))

We want to save our output in the CSV file.

In [ ]:
result = pd.DataFrame({'id': test1.id.values, 'price_value': col0
                       , 'fake_originality':col1, 'warranty':col2,
                       'size':col3,'discrepancy':col4,
                       'flavor_odor':col5, 'expiration_date':col6})

result.to_csv('result_comment3.csv', index=False)

In [ ]:
result = pd.read_csv('/content/result_comment3.csv')

In [ ]:
result.head()

,id,price_value,fake_originality,warranty,size,discrepancy,flavor_odor,expiration_date
0,1130083,0,0,0,0,0,0,0
1,1051036,0,0,0,0,0,0,0
2,1240179,1,0,0,0,0,0,0
3,1197397,0,0,0,0,1,0,0
4,1212378,0,0,0,0,0,0,0


In [ ]:
result.shape

(56033, 8)

This model gives us 85% accuracy in the contest.